Set-Up

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))

from scripts import download
from scripts.constants import Constants
from scripts.table.table_constants import TableConstants
from scripts.table.table_prompt import TablePrompts
from scripts.llm.llm_request import ParallelAPIRequester
from scripts.evaluation import stats
from scripts.table import table_extraction, table_types, table_stats
from scripts.llm import llm_stats

project_path = os.path.abspath(os.path.expandvars(Constants.PROJECT_PATH))

experiments_path = os.path.join(project_path, Constants.Directories.EXPERIMENTS)

extracted_tables_path = os.path.join(experiments_path, Constants.Directories.EXTRACTED_TABLE)
config_path = os.path.join(project_path, Constants.Directories.CONFIGURATIONS)
comparison_path = os.path.join(experiments_path, Constants.Directories.COMPARISONS)
ground_truth_path = os.path.join(experiments_path, Constants.Directories.GROUND_TRUTH)

Download Articles

In [ ]:
article_dir = "CS_Test_Small"
articles_path = os.path.join(experiments_path, Constants.Directories.ARTICLES, article_dir)

json_file_name = f"{article_dir}.json"
extracted_tables_file_path = os.path.join(extracted_tables_path, json_file_name)

# Download articles as HTML files
# search_query = '"Entity Matching"'
# download.get_articles(search_query, articles_path)

In [ ]:
table_extraction.extract_and_save_tables(articles_path, extracted_tables_file_path)

In [ ]:
table_extraction.check_extracted_data(os.path.join(extracted_tables_path, json_file_name))

Claim Extraction

In [ ]:
output_dir = "cs_test_small_standard_prompt"
tables_file_name = json_file_name
msgs_dir = "CS_Standard"

request_path = os.path.join(experiments_path, Constants.Directories.OUTPUT, output_dir)
msgs_path = os.path.join(project_path, Constants.Directories.MESSAGES, msgs_dir)

connection_info_path = os.path.join(config_path, "connection_info.json")
request_conf_path = os.path.join(
    config_path, Constants.Directories.API, "standard_parallel.json" 
)

In [ ]:
table_prompts = TablePrompts(request_path, extracted_tables_file_path, msgs_path)
prompts = table_prompts.generate()

In [ ]:
llm = ParallelAPIRequester(
    connection_info_path,
    request_conf_path
)

results = llm.run(
    request_path,
    prompts
)

Ground Truth

In [ ]:
gt_answer_path = os.path.join(ground_truth_path, "claim_labels")

# gt_file = os.path.join(ground_truth_path, "gt_labeled.ods")
# stats.write_ground_truth(gt_file, gt_answer_path)

In [ ]:
comp_dir_name = "claim_types_standard_prompt"
comparison_result_path = os.path.join(comparison_path, comp_dir_name)

In [ ]:
table_types.check_claims_types(gt_answer_path, request_path, comparison_result_path)

# request_paths = [
#     request_path
# ]

# output_types = TableConstants.Types.CLAIMED_TYPES

# table_stats.compare_multiple_results(
#     ground_truth_path=gt_answer_path,
#     request_paths=request_paths,
#     comparison_path=comparison_result_path
# )